In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:

# Data Augmentation for training set
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
)

# Load in the training dataset with augmentation applied
train_generator = train_datagen.flow_from_directory(
    '../binary-brain-classification/training',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)

# Only rescale the test set
test_datagen = ImageDataGenerator(
    rescale = 1./255
)

# Load in the test dataset with only rescaling
test_generator = test_datagen.flow_from_directory(
    '../binary-brain-classification/testing',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 3660 images belonging to 2 classes.
Found 392 images belonging to 2 classes.


In [3]:
model = keras.Sequential([

    # Define the input shape
    keras.layers.Input(shape=(224,224,3)),
    
    # CNN layer 1
    keras.layers.Conv2D(16, (3,3), activation=None),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling2D((2,2)),
    
    # Flatten and classify
    keras.layers.Flatten(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation = 'sigmoid')

])
model.summary()

2025-01-09 12:27:59.789721: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-01-09 12:27:59.789755: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-01-09 12:27:59.789763: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-01-09 12:27:59.789781: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-09 12:27:59.789796: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 222, 222, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 197136)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 197136)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │       197,137 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 197,649 (772.07 KB)

 Trainable params: 197,617 (771.94 KB)

 Non-trainable params: 32 (128.00 B)

In [4]:
# Compile the Model
model.compile(
    optimizer = 'adam',    # Popular optimizer
    loss = 'binary_crossentropy',   # Categorical crossentropy for classification tasks
    metrics = (['accuracy'])    # Measure the accuracy
)

In [5]:
# Run with GPU for better performance
with tf.device('/GPU:0'):   
    # Test the model
    model.fit(
        train_generator,    # Traning dataset from our generator
        epochs = 10    # See all the images 10 times
    )

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


2025-01-09 12:28:00.193682: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


115/115 ━━━━━━━━━━━━━━━━━━━━ 9s 74ms/step - accuracy: 0.8045 - loss: 1.9083
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 8s 73ms/step - accuracy: 0.8899 - loss: 0.8556
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 8s 72ms/step - accuracy: 0.9369 - loss: 0.4543
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 8s 72ms/step - accuracy: 0.9593 - loss: 0.2734
Epoch 5/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 8s 72ms/step - accuracy: 0.9732 - loss: 0.2106
Epoch 6/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 8s 72ms/step - accuracy: 0.9862 - loss: 0.0762
Epoch 7/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 8s 72ms/step - accuracy: 0.9796 - loss: 0.1048
Epoch 8/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 8s 72ms/step - accuracy: 0.9896 - loss: 0.0339
Epoch 9/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 8s 73ms/step - accuracy: 0.9907 - loss: 0.0674
Epoch 10/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 8s 73ms/step - accuracy: 0.9904 - loss: 0.0530


In [6]:
model.evaluate(test_generator)

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.8845 - loss: 1.9182


[1.4054574966430664, 0.8877550959587097]

In [7]:
model.save('./models/binary_model.keras')